<a href="https://colab.research.google.com/github/marcioluiskroth/imersao_ai_alura/blob/main/AIdeaBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instala a biblioteca google-generativeai
!pip install -q -U google-generativeai

# Importa a biblioteca e configura a chave API
import google.generativeai as genai
from google.colab import userdata
import random

minha_api = userdata.get('SECRET_KEY')
genai.configure(api_key=minha_api)

# Define o modelo de linguagem
model = genai.GenerativeModel('gemini-1.5-pro-latest')

def get_user_input(prompt, is_required=True):
    user_input = input(prompt)
    while is_required and user_input.strip() == "":
        print("Este campo é obrigatório. Por favor, forneça uma resposta.")
        user_input = input(prompt)
    return user_input

def start_conversation():
    action_verbs = ["crie", "pesquise", "gere", "escreva", "analise", "apresente", "descreva"]
    print(f"Por favor, descreva a tarefa que você deseja que o chatbot execute. Você pode usar verbos como {', '.join(random.sample(action_verbs, 3))}.")
    task = get_user_input("Tarefa a ser realizada pelo chatbot: ")

    print("\nExemplos de contextos: 1) Criar um resumo de um livro. 2) Pesquisar as últimas notícias sobre tecnologia. 3) Gerar um código para análise de dados.")
    context = get_user_input("Contexto relacionado à tarefa: ")

    example_text = get_user_input("Texto de exemplo (opcional): ", is_required=False)

    print("\nComo o chatbot deve agir? Exemplos: 1) Como um assistente amigável. 2) Como um professor. 3) Como um consultor técnico.")
    persona = get_user_input("Como deve atuar o chatbot: ")

    format = get_user_input("Formato do retorno (ex: tabela, texto, código), opcional: ", is_required=False)

    print("\nExemplos de tom: 1) Profissional. 2) Respeitoso. 3) Sarcástico.")
    humor = get_user_input("Tom da resposta (opcional): ")

    preprompt = f" Atue como {persona}, com um humor {humor}, {task}, no formato de {format}, dentro do seguinte contexto: {context}, {example_text}"
    return preprompt

def confirm_prompt(preprompt):
    print(f"Acho que entendi o que você precisa que eu faça, seria isso? \"{preprompt}\"")
    while True:
        user_choice = input("Posso continuar? [Sim/Não/Editar/Recomeçar]: ").lower()
        if user_choice == "sim":
            return True, preprompt
        elif user_choice == "não":
            return False, preprompt
        elif user_choice == "editar":
            edited_prompt = get_user_input("Edite a frase como desejar: ")
            return True, edited_prompt
        elif user_choice == "recomeçar":
            return None, None

def send_to_api(prompt):
    try:
        instruction = f" {prompt}"
        chat = model.start_chat(history=[])
        response = chat.send_message(instruction)
        return response.text
    except google.generativeai.StopCandidateException as e:
        print("Desculpe, o conteúdo fornecido foi considerado inapropriado pela API. Por favor, reformule sua solicitação.")
        return None

def main():
    while True:
        preprompt = start_conversation()
        continuation, temp_prompt = confirm_prompt(preprompt)
        if continuation is True:
            final_response = send_to_api(temp_prompt)
            if final_response:
                print(f"Resposta final: {final_response}\n\n")
                more = input("Algo mais? [Sim/Não]: ").lower()
                if more == "não":
                    break
            else:
                print("Vamos tentar novamente.")
                continue
        elif continuation is False:
            break
        elif temp_prompt is None:
            continue

# Inicia a interação
if __name__ == "__main__":
    main()


Por favor, descreva a tarefa que você deseja que o chatbot execute. Você pode usar verbos como gere, crie, pesquise.
